#### Cross Validation

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

#### K- Fold Cross Validation

In [6]:
x = [1,2,3,4,5,6,7,8,9]
from sklearn.model_selection import KFold
kf = KFold(n_splits = 3)

In [7]:
#1,2,3,4,5,6,7,8,9
#1 2 3 4 5 6 7 8 9
#[1 2 3 4 5 6 7 8 9][9]
#[1 2 3 4 5 6 7 8 9][8]
for i,j in kf.split(x):
    print(i, j)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [10]:
from sklearn.datasets import load_iris
iris = load_iris()

In [12]:
df = pd.DataFrame(iris['data'],columns=['SL','SW','PL','PW'])
df['target'] = iris['target']
df.head()

,SL,SW,PL,PW,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [14]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(df[['SL','SW','PL','PW']],df['target'])
xtr.head()

,SL,SW,PL,PW
101,5.8,2.7,5.1,1.9
22,4.6,3.6,1.0,0.2
132,6.4,2.8,5.6,2.2
75,6.6,3.0,4.4,1.4
103,6.3,2.9,5.6,1.8


In [19]:
dfTr = pd.concat([xtr,ytr],axis = 1)
dfTs = pd.concat([xts,yts],axis = 1)
# dfTr

In [22]:
def hitungScore(model,xtr,xts,ytr,yts):
    model.fit(xtr,ytr)
    return model.score(xts,yts)

In [29]:
score = []
kf = KFold(n_splits = 5)
for i,j in kf.split(np.array(dfTr)):
    #print(i,j) # i = index train, j= index validation
    #print(dfTr.iloc[i])
    xtrCv = dfTr.iloc[i][['SL','SW','PL','PW']]
    ytrCv = dfTr.iloc[i]['target']
    xValCv = dfTr.iloc[j][['SL','SW','PL','PW']]
    yValCv = dfTr.iloc[j]['target']
    score.append(
    hitungScore(LogisticRegression(),xtrCv,xValCv,ytrCv,yValCv))
np.mean(score)
#dftr = 135 => 5 fold => 27
# training 27 + 27 + 27  + 27 =  validation = 27

0.9553359683794467

<hr>

In [30]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(LogisticRegression(),dfTr[['SL','SW','PL','PW']],
                      dfTr['target'],cv=5))

print(np.mean(cross_val_score(LogisticRegression(),dfTr[['SL','SW','PL','PW']],
                        dfTr['target'],cv=5,scoring='accuracy')))

[1.         0.91304348 0.95454545 0.90909091 1.        ]
0.9553359683794467


In [31]:
from sklearn.neighbors import KNeighborsClassifier
model1 = KNeighborsClassifier(n_neighbors=3)
model2 = KNeighborsClassifier(n_neighbors=4)

In [33]:
score1= cross_val_score(model1,dfTr[['SL','SW','PL','PW']],
                      dfTr['target'],cv=5,scoring='balanced_accuracy')
print(score1)
print(score1.mean())


[1.         0.96296296 0.95833333 0.94444444 1.        ]
0.9731481481481481


In [34]:
score2 = cross_val_score(model2,dfTr[['SL','SW','PL','PW']],
                      dfTr['target'],cv=5,scoring='balanced_accuracy')
print(score2)
print(score2.mean())

[1.         0.96296296 0.88888889 0.94444444 1.        ]
0.9592592592592591


#### Training => train & validation:
- K- fold, Repeated, Stratisfied
- Shuffle Split, Stratified Shuffle Split
- Leave One Out => K-Fold 1 => 1,2,3,4 => 1 2 3 4 => 123 4
- Leave P out

<hr>


### Hyper-parameter Tuning

In [35]:
df1 = pd.DataFrame(iris['data'],columns=['SL','SW','PL','PW'])
df1['target'] = iris['target']
df1.head()

,SL,SW,PL,PW,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [39]:
x_train,x_test,y_train,y_test = train_test_split(df1[['SL','SW','PL','PW']],df1['target'])


In [40]:
model3 = LogisticRegression()
model3.fit(x_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
model3.score(x_test,y_test)

0.9736842105263158

In [42]:
print(cross_val_score(model3,x_train,y_train))
print(np.mean(cross_val_score(model3,x_train,y_train)))

[0.86956522 0.95652174 1.         1.         1.        ]
0.9652173913043478


In [43]:
# param yang akan diTuned + nilai yang mungkin
penalty = ['l1','l2','elasticnet','none']
solver = ['newton-cg','lbfgs','liblinear','sag','saga']
max_iter = [1,10,100,1000,10000]
param = {'penalty':penalty,'solver':solver,'max_iter':max_iter}
param

{'penalty': ['l1', 'l2', 'elasticnet', 'none'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
 'max_iter': [1, 10, 100, 1000, 10000]}

<hr>


Hyperparameter Tuning :
- Randomized search Cross Validation
- Grid Search Cross Valiation = 4*5*5 = 100

#### Randomized search CV

In [44]:
from sklearn.model_selection import RandomizedSearchCV
model4 = RandomizedSearchCV(
    estimator = model3,param_distributions = param, cv=5)

In [46]:
model4.fit(x_train,y_train)
model4.best_params_

{'solver': 'saga', 'penalty': 'l1', 'max_iter': 100}

In [47]:
model3.score(x_test,y_test)

0.9736842105263158

In [49]:
# improve testing with params
modelBaru = LogisticRegression(solver='saga',penalty='l1',max_iter=100)
modelBaru.fit(x_train,y_train)
modelBaru.score(x_test,y_test)


1.0

### Grid Search CV

In [50]:
from sklearn.model_selection import GridSearchCV
model5 = GridSearchCV(model3, param, cv=5)


In [52]:
model5.fit(x_train,y_train)
model5.best_params_


{'max_iter': 10, 'penalty': 'none', 'solver': 'newton-cg'}

In [53]:
model3.score(x_test,y_test)

0.9736842105263158

In [54]:
model5baru = LogisticRegression(max_iter=10,penalty='none',solver='newton-cg')
model5baru.fit(x_train,y_train)
model5baru.score(x_test,y_test)

1.0